In [114]:
#Kütüphanelerin eklenmesi
import numpy as np #Bu kütüphane lineer cebir için kullandığımız kütüphane fonksiyonlarını içeriyor
import pandas as pd # verilerimizi işlemek için pandas kütüphanasini kullanıyoruz(örn pd.read_scv)
from sklearn.metrics import *
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold, train_test_split, cross_val_predict, cross_val_score
import matplotlib.pyplot as plt
%matplotlib inline
!pip install utils
from utils import *
import json
import random
from spacy.tokenizer import Tokenizer
from spacy.lang.tr import Turkish
import warnings
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
import re
from snowballstemmer import TurkishStemmer
warnings.filterwarnings('ignore') 
import string
from sklearn.pipeline import make_pipeline

from snowballstemmer import TurkishStemmer

import xlsxwriter 
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
import time
from spacy.lang.tr import Turkish
from sklearn.metrics import classification_report, accuracy_score, make_scorer,precision_score,recall_score,f1_score

You should consider upgrading via the 'd:\program\anaconda\python.exe -m pip install --upgrade pip' command.


In [115]:
column = ['tweets','duygu']
df = pd.read_excel("../dataset/kemik_pos_neg.xlsx")
df.columns=column
#veri setinin gösterilmesi
df=df.drop_duplicates()

In [116]:
len(df[df.duygu=='olumlu']),len(df[df.duygu=='olumsuz'])

(4563, 4563)

In [117]:
# df.loc[df['duygu'] == "olumlu", "duygu"] = 1
# df.loc[df['duygu'] == "olumsuz", "duygu"] = 0

In [118]:
from typing import List
from jpype import JClass, JString, getDefaultJVMPath, shutdownJVM, startJVM, java

In [119]:
ZEMBEREK_PATH = r'C:\Users\asus\Desktop\Python\word2vec\zemberek-full.jar'
startJVM(getDefaultJVMPath(), '-ea', '-Djava.class.path=%s' % (ZEMBEREK_PATH))

In [120]:
nlp= Turkish()
from zemberek import (
    TurkishSentenceNormalizer,
    TurkishMorphology,
)
morphology = TurkishMorphology.create_with_defaults() 

normalizer = TurkishSentenceNormalizer(morphology) #kelimelerin düzgün yazımı için

TurkishMorphology = JClass('zemberek.morphology.TurkishMorphology')
morphology2 = TurkishMorphology.createWithDefaults()

In [121]:
#verilerde olan stop wordleri atmak için yapılan işlemler
#veriler için 3 ayrı dataset birleştirilip sayısallaştırılmış, küçük harfe çevrilmiş ve noktalama işaretlerinden arındırılmıştır.
workbook = xlsxwriter.Workbook('C:/Users/asus/Desktop/Python/tf-idf/preprocessing.xlsx') 
worksheet = workbook.add_worksheet()

def remove(string_input):
    return "".join(string_input.split()) 

def remove_stopwords(df_fon):
    i=0
    print(df_fon['tweets'].size)
    while i <(df_fon['tweets'].size):
        v=""
        for token in nlp(df_fon['tweets'][i]):
            yeniMetin=""
            #spacy kütüphanesini kullanarak noktalama işaretlerinden kurtardım
            yeniMetin = token.text.lower() # spacy kütüphanesiyle küçük harfe çevirip token ayırdım
            yeniMetin =remove(yeniMetin)
            word=""
            if yeniMetin not in string.punctuation:
                word += yeniMetin          
            
            if word not in ['',' ','olurmusunuz','alıyormusunuz','musunuz','biliyormusunuz','ediyormusunuz?']:
                word = normalizer.normalize(word) #cümleleri düzgün hale getiriyor. yazım hataları giderildi
                if word not in stopwords.words('turkish'):
                    word = normalizer.normalize(word) #cümleleri düzgün hale getiriyor. yazım hataları giderildi
                    analysis: java.util.ArrayList = (morphology2.analyzeAndDisambiguate(word).bestAnalysis())
                    pos: List[str] = []
                    for j, analysis in enumerate(analysis, start=1):
                        pos.append(f'{str(analysis.getLemmas()[0])}')
                    
                    v+= f' {"".join(pos)}'
        print(v)
        
        column = 0
                
        worksheet.write(i, column, v) 

        i += 1
    workbook.close() 

remove_stopwords(df)